In [1]:
import numpy as np
import igl
import meshplot as mp
import scipy

# Reading point cloud

In [2]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

/Users/yinuochang/miniforge3/envs/gm2022/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting up the Constraints

In [3]:
# use KD-Tree to find the nearest neigbor
from scipy.spatial import KDTree
tree = KDTree(pi, leafsize=pi.shape[0]+1)
def find_closed_point(point, points):
    distances, ndx = tree.query([point], k=2)
    return ndx[0,1]     

In [4]:
def setUpConstraints(pi,ni):
    #use to store the pis and target values
    #*3 for f(pi) f(pi+) f(pi-)
    size = pi.shape[0]
    constrained_pi, target_v = np.zeros((size* 3, 3)), np.zeros((size * 3))
    for i in range(size):
        #Fix an eps value, for instance eps = 0.01 bounding_box_diagonal
        eps = 0.01 * igl.bounding_box_diagonal(pi)
        eps_plus = eps
        eps_minus = eps
        #For each point pi in the point cloud, add a constraint of the form f(pi) = di = 0.
        constrained_pi[i] = pi[i]
        target_v[i] = 0
        #For each point pi compute pi+ = pi + eps ni, where ni is the normalized normal of pi. Check that pi is the closest point to pi+ if not, halve esp and recompute pi+ until this is the case. Then, add another constraint equation: f(pi+) = eps.
        constrained_pi[i + size] = pi[i] + eps * ni[i]
        if find_closed_point(constrained_pi[i + size],pi) != i:
            eps_plus = 0.5 * eps
        constrained_pi[i + size] = pi[i] + eps_plus * ni[i]
        target_v[i + size] =  eps_plus
        
        #Repeat the same process for -eps, i.e., add equations of the form f(pi-) = -eps. Do not forget to check each time that pi is the closest point to pi-.
        constrained_pi[i + 2*size] = pi[i] - eps * ni[i]
        if find_closed_point(constrained_pi[i + 2*size],pi) != i:
            eps_minus = 0.5 * eps
        constrained_pi[i + 2*size] = pi[i] - eps_minus * ni[i]
        target_v[i + 2*size] =  -eps_minus
    return constrained_pi, target_v
        

In [5]:
constrained_pi, target_v = setUpConstraints(pi, ni)

In [6]:
color_map = np.zeros(constrained_pi.shape)
size = pi.shape[0]
color_map[0:size] = np.array([0, 0, 1])
color_map[size:size*2] = np.array([1, 0, 0])
color_map[size*2:size*3] = np.array([124/255, 252/255, 0])
a = mp.plot(pi, c = np.zeros((pi.shape[0], 3)), shading={"point_size": 8})
mp.plot(constrained_pi, c = color_map, shading = {"point_size": 8})

/Users/yinuochang/miniforge3/envs/gm2022/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0060472…

# Use MLS interpolation to extend to function f

## Create a grid sampling the 3D space

In [7]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

## MLS Interpolation

Not using explict method. At a given point xi in x, you evaluate this function by finding the "optimal" basis function coefficients (which will vary from point to point!) and using these to combine the basis function values at xi.

In [8]:
def wendland(r, h): #wr: 19:31
    if r<h:
        return ((1 - r / h)** 4) * (4 * r / h + 1)
    else:
        return 0

In [9]:
def polyDegree(degree,x):
    # x is the grid vertices
    basis =  {
        0 : np.array([1]),
        1 : np.array([1, x[0], x[1], x[2]]),
        2 : np.array([1, x[0], x[1], x[2], 
                      x[0] * x[1], x[1] * x[2], x[0] * x[2], 
                          x[0] ** 2, x[1] ** 2, x[2] ** 2])
    }
    return basis.get(degree, "Set Wrong")
    

In [10]:
def closest_points(point, points, h):
    #retreives the indices all points in points that are at distance less than h from point.
    res = []
    x = point[0]
    y = point[1]
    z = point[2]
    for i in range(points.shape[0]):
        relx = points[i][0]
        rely = points[i][1]
        relz = points[i][2]
        r2 =  (relx -x)**2 + (rely -y)**2 + (relz -z)**2
        if r2 < h**2:
            res.append(i)
    return res

In [11]:
closest_points(pi[0],pi,1)

[0]

# MLS function

In [12]:
# Parameters
bbox_min = np.min(pi, axis = 0)
bbox_max = np.max(pi, axis = 0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

## construct an implicit function for MLS

In [13]:
#xi: point; wendlandRadius is also the h in closest_points; constrained_pi[index_after_closed] is p
def MLSImplicitFuction(xi, wendlandRadius, poly_degree, constrained_pi, target_v):
    if poly_degree == 0:
        coef = 1
    if poly_degree == 1:
        coef = 4
    if poly_degree == 2:
        coef = 10
    iLessh = closest_points(xi, constrained_pi, wendlandRadius)
#     if the number of constraint points within wendlandRadius is less than twice the number of polynomial coefficients, 
# you can assign a large positive (outside) value to the grid point.
    if len(iLessh) < 2 * coef:
        return 100000
    else:
        #(BT * W * B)a(x) = BT*W*d
        # W = wendland(||xi-c||) norm!!!
        # return poly*a(x)
        B = np.zeros((len(iLessh), coef))
        W = np.eye(len(iLessh))
        d = target_v[iLessh]
        p = constrained_pi[iLessh]
        
        for i in range(len(iLessh)):
            W[i,i] = wendland(np.linalg.norm(xi - p[i]), wendlandRadius)
            B[i] = polyDegree(poly_degree, p[i])
        
        ax = np.linalg.solve(((B.T).dot(W)).dot(B),((B.T).dot(W)).dot(d))
        return polyDegree(poly_degree, xi).dot(ax)



In [14]:
# Generate grid n x n x n
# grid vertices x and the tets connecting them T
x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = bbox_diag * 0.1
k = 1
# fx = np.linalg.norm(x-center, axis=1) - radius  #This is the initial function we need to change it to the implict one
fx = np.array([MLSImplicitFuction(xi, radius, k, constrained_pi, target_v) for xi in x])

In [16]:
# Treshold fx to visualize inside outside
color = np.zeros_like(x)
color[fx < 0] = np.array([1, 0, 0])
color[fx >= 0] = np.array([124/255, 252/255, 0])
# ind = np.zeros_like(fx)
# ind[fx >= 0] = 1
# ind[fx < 0] = -1
mp.plot(x, c=color, shading={"point_size": 3,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

## Implementing a spatial index to accelerate neighbor calculations
### use spatial trees to split the space x ??
### use regular grid to recalculate

In [17]:
def set_newgrid(step, p):
    mmax = np.max(p, axis = 0)
    mmin = np.min(p, axis = 0)
    dim = mmax - mmin
    diag = np.linalg.norm(dim)
    grid_step = step * diag
    units_size = np.zeros(3)
    units_size = np.ceil(dim / grid_step)
    newgrid_size = int(np.prod(units_size))
    newgrid = [[] for i in range(newgrid_size)]
    
    for i in range(p.shape[0]):
        p_dist = np.zeros(3)
        p_dist = np.floor((p[i] - mmin) / grid_step)
        index = int(p_dist[0] + units_size[0] * p_dist[1] + units_size[0] * units_size[1] * p_dist[2])
        newgrid[index].append(i)
    
    return newgrid, units_size, grid_step

In [18]:
step = 0.1
newgrid, units_size, grid_step = set_newgrid(step, pi)

In [19]:
def acc_find_closed_point(p, points, newgrid, units_size, grid_step):
    closest_i = 0
    min_val = 10000
    mmin = np.min(points, axis = 0)
    p_dist = np.zeros(3)
    p_dist = np.floor((p - mmin) / grid_step)

    check_distance = 1 
#     min_Block_offset = np.ceil(p_dist - check_distance)
#     min_Block_offset[min_Block_offset<0] = 0
    min_Block_offset_x = int(max(0, p_dist[0] - check_distance))
    min_Block_offset_y = int(max(0, p_dist[1] - check_distance))
    min_Block_offset_z = int(max(0, p_dist[2] - check_distance))
    max_Block_offset_x = int(min(p_dist[0] + 1 + check_distance, units_size[0]))
    max_Block_offset_y = int(min(p_dist[1] + 1 + check_distance, units_size[1]))
    max_Block_offset_z = int(min(p_dist[2] + 1 + check_distance, units_size[2]))

   # find out if there's better solution ? No need not O(n^4)
    for i in range(min_Block_offset_x, max_Block_offset_x):
        for j in range(min_Block_offset_y, max_Block_offset_y):
            for k in range(min_Block_offset_z, max_Block_offset_z):
                index = int(i + units_size[0] * j + units_size[0] * units_size[1] * k)
                for q in range(len(newgrid[index])):
                    distance = np.linalg.norm(p - points[newgrid[index][q]])
                    if distance < min_val:
                        closest_i = newgrid[index][q]
                        min_val = distance
    return closest_i

In [20]:
def acc_closest_points(p, points, h, newgrid, units_size, grid_step):
    dist_index = []
    mmin = np.min(points, axis = 0)
    n_ori_points = int(points.shape[0] / 3)
    p_dist = np.zeros(3)
    p_dist = np.floor((p - mmin) / grid_step)

    check_distance = 1
    min_Block_offset_x = int(max(0, p_dist[0] - check_distance))
    min_Block_offset_y = int(max(0, p_dist[1] - check_distance))
    min_Block_offset_z = int(max(0, p_dist[2] - check_distance))
    max_Block_offset_x = int(min(p_dist[0] + 1 + check_distance, units_size[0]))
    max_Block_offset_y = int(min(p_dist[1] + 1 + check_distance, units_size[1]))
    max_Block_offset_z = int(min(p_dist[2] + 1 + check_distance, units_size[2]))
    
    for i in range(min_Block_offset_x, max_Block_offset_x):
        for j in range(min_Block_offset_y, max_Block_offset_y):
            for k in range(min_Block_offset_z, max_Block_offset_z):
                index = int(i + units_size[0] * j + units_size[0] * units_size[1] * k)
                for q in range(len(newgrid[index])):
                    
                    distance = np.linalg.norm(p - points[newgrid[index][q]])
                    if distance < h:
                        dist_index.append(newgrid[index][q])
                        
                    distance = np.linalg.norm(p - points[n_ori_points + newgrid[index][q]])
                    if distance < h:
                        dist_index.append(n_ori_points + newgrid[index][q])
                        
                    distance = np.linalg.norm(p - points[n_ori_points * 2 + newgrid[index][q]])
                    if distance < h:
                        dist_index.append(n_ori_points * 2 + newgrid[index][q])
                    
    return dist_index

In [21]:
def acc_setUpConstraints(pi,ni,newgrid, units_size, grid_step):
    #use to store the pis and target values
    #*3 for f(pi) f(pi+) f(pi-)
    size = pi.shape[0]
    constrained_pi, target_v = np.zeros((size* 3, 3)), np.zeros((size * 3))
    for i in range(size):
        #Fix an eps value, for instance eps = 0.01 bounding_box_diagonal
        eps = 0.01 * igl.bounding_box_diagonal(pi)
        eps_plus = eps
        eps_minus = eps
        #For each point pi in the point cloud, add a constraint of the form f(pi) = di = 0.
        constrained_pi[i] = pi[i]
        target_v[i] = 0
        #For each point pi compute pi+ = pi + eps ni, where ni is the normalized normal of pi. Check that pi is the closest point to pi+ if not, halve esp and recompute pi+ until this is the case. Then, add another constraint equation: f(pi+) = eps.
        constrained_pi[i + size] = pi[i] + eps * ni[i]
        if acc_find_closed_point(constrained_pi[i + size],pi, newgrid, units_size, grid_step) != i:
            eps_plus = 0.5 * eps
        constrained_pi[i + size] = pi[i] + eps_plus * ni[i]
        target_v[i + size] =  eps_plus
        
        #Repeat the same process for -eps, i.e., add equations of the form f(pi-) = -eps. Do not forget to check each time that pi is the closest point to pi-.
        constrained_pi[i + 2*size] = pi[i] - eps * ni[i]
        if acc_find_closed_point(constrained_pi[i + 2*size],pi, newgrid, units_size, grid_step) != i:
            eps_minus = 0.5 * eps
        constrained_pi[i + 2*size] = pi[i] - eps_minus * ni[i]
        target_v[i + 2*size] =  -eps_minus
    return constrained_pi, target_v
        

In [22]:
def acc_MLSImplicitFuction(xi, wendlandRadius, poly_degree, constrained_pi, target_v, newgrid, units_size, grid_step):
    if poly_degree == 0:
        coef = 1
    if poly_degree == 1:
        coef = 4
    if poly_degree == 2:
        coef = 10
    iLessh = acc_closest_points(xi, constrained_pi, wendlandRadius, newgrid, units_size, grid_step)
#     if the number of constraint points within wendlandRadius is less than twice the number of polynomial coefficients, 
# you can assign a large positive (outside) value to the grid point.
    if len(iLessh) < 2 * coef:
        return 10000
    else:
        #(BT * W * B)a(x) = BT*W*d
        # W = wendland(||xi-c||) norm!!!
        # return poly*a(x)
        B = np.zeros((len(iLessh), coef))
        W = np.eye(len(iLessh))
        d = target_v[iLessh]
        p = constrained_pi[iLessh]
        
        for i in range(len(iLessh)):
            W[i,i] = wendland(np.linalg.norm(xi - p[i],2), wendlandRadius)
            B[i] = polyDegree(poly_degree, p[i])
        
        ax = np.linalg.solve(((B.T).dot(W)).dot(B),((B.T).dot(W)).dot(d))
        return polyDegree(poly_degree, xi).dot(ax)

In [23]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)

In [24]:
step = 0.1
newgrid, units_size, grid_step = set_newgrid(step, pi)

In [25]:
constrained_p, constrained_v = acc_setUpConstraints(pi, ni, newgrid, units_size, grid_step)

In [26]:
bbox_max = np.max(pi, axis = 0)
bbox_min = np.min(pi, axis = 0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
n = 10
x, T = tet_grid((n,n,n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

In [27]:
k = 1
wendlandRadius = bbox_diag * 0.1
fx = np.array([acc_MLSImplicitFuction(xi, wendlandRadius, k, constrained_p, constrained_v, newgrid, units_size, grid_step) for xi in x])
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
color = np.zeros_like(x)
color[fx < 0] = np.array([1, 0, 0])
color[fx >= 0] = np.array([124/255, 252/255, 0])
mp.plot(x, c=color, shading={"point_size": 4, "width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Extracting the surface

In [28]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.6844348…

In [29]:
def filterF(face):
    f = igl.face_components(face)
    components = []
    for i in range(f.shape[0]):
        if f[i] not in components:
            components.append(f[i])
    connected_components = np.zeros(len(components))
    for i in range(f.shape[0]):
        connected_components[f[i]] += 1
    return face[f == np.argmax(connected_components)]

In [30]:
mp.plot(sv, filterF(sf), shading={"wireframe": True}) 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.6844348…

In [31]:
# Experiment with different parameter settings: grid resolution (also anisotropic in the 3 axes), Wendland function radius, and polynomial degree.
# change radius by changing the multiplier of bbox_diag
# Para we have calculated:constrained_pi, target_v, pi,ni
def extract_surface(data,res_x, res_y, res_z, polyDegree, diagRate, step, filter = True):
    pi, v = igl.read_triangle_mesh(data)
    pi /= 10
    ni = igl.per_vertex_normals(pi, v)
    newgrid, units_size, grid_step = set_newgrid(step, pi)
    constrained_p, constrained_v = acc_setUpConstraints(pi, ni, newgrid, units_size, grid_step)
    bbox_max = np.max(pi, axis = 0)
    bbox_min = np.min(pi, axis = 0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
    wendlandRadius = bbox_diag * diagRate
    x, T = tet_grid((res_x, res_y, res_z), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)  
    fx = np.array([acc_MLSImplicitFuction(xi, wendlandRadius, polyDegree, constrained_p, constrained_v, newgrid, units_size, grid_step) for xi in x])    
    sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
    if filter:
        mp.plot(sv, filterF(sf), shading={"wireframe": True})       
    else:
        mp.plot(sv, sf, shading={"wireframe": True})

In [32]:
extract_surface("data/cat.off", 20, 20, 20, 2, 0.1,0.1, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.4346818…

In [33]:
extract_surface("data/cat.off", 30, 40, 35, 1, 0.2,0.1, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(9.1739492…

In [34]:
extract_surface("data/cat.off", 20, 35, 30, 1, 0.1,0.1, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.4415092…

# Using a non-axis-aligned grid.
The point cloud luigi.off is not aligned with the canonical axes. Running reconstruction on an axis-aligned grid is wasteful in this case: many of the grid points will lie far outside the object. Devise an automatic (and general) way to align the grid to the data and implement it. 

Required output of this section:  
Plot of the grid with nodes colored according to their implicit function values.

In [35]:
#PCA？
# Read Data
p1, v1 = igl.read_triangle_mesh("data/luigi.off")
p1 /= 10
n1 = igl.per_vertex_normals(p1, v1)
mp.plot(p1, shading={"point_size": 0.6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

In [36]:
def PCA(points,ni):
    points -= np.mean(points, axis = 0)  
    cov = np.cov(points, rowvar = False)
    evals , evecs = scipy.linalg.eigh(cov)
    idx = np.argsort(evals)[::-1]
    evecs = evecs[:,idx]
    evals = evals[idx]
    return np.dot(points, evecs), np.dot(ni,evecs)

In [37]:
new_p1, new_n1 = PCA(p1,n1)

In [38]:
step = 0.1
newgrid, units_size, grid_step = set_newgrid(step, new_p1)
constrained_p1, constrained_v1 = acc_setUpConstraints(new_p1, new_n1,newgrid, units_size, grid_step)

In [39]:
bbox_max = np.max(new_p1, axis = 0)
bbox_min = np.min(new_p1, axis = 0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
n=30
x1, T1 = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

In [40]:
k = 1
wendlandRadius = bbox_diag * 0.1
fx = np.array([acc_MLSImplicitFuction(x1i, wendlandRadius, k, constrained_p1, constrained_v1, newgrid, units_size, grid_step) for x1i in x1])
# ind = np.zeros_like(fx)
# ind[fx >= 0] = 1
# ind[fx < 0] = -1
color = np.zeros_like(x1)
color[fx < 0] = np.array([1, 0, 0])
color[fx >= 0] = np.array([124/255, 252/255, 0])
mp.plot(x1, c=color, shading={"point_size": 0.5, "width": 800, "height": 800})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.667772…

# Screened Poisson Surface Reconstruction
The picture generated by meshlab is shown below.
It is more smooth with two axes and automatically fills holes

In [47]:
%%html
<img src='./meshlabcat.png', width=600, height=600>

In [48]:
%%html
<img src='./meshlabluigi.png', width=600, height=600>